# 1.8e: Unreachable Token Extraction

This notebook extracts and saves the set of unreachable tokens for downstream analysis.

## The Question

In 1.8d, we discovered:
- **3,357 total unreachable tokens** in the vocabulary
- **1,934 overlap with cluster** (87.4% of cluster)
- **1,423 unreachable tokens OUTSIDE the cluster**

**Hypothesis:** The 1,423 form a diffuse halo around the cluster—tokens that escaped the core singularity via early stochastic gradient kicks (bfloat16 cell-hopping) but remained in orbit due to lack of differential training signal.

## Method

Load 1.8d results and save clean masks/IDs for:
1. **All unreachable tokens** (3,357)
2. **Unreachable halo** (1,423 outside cluster)
3. **Cluster** (2,212 from 1.4h)

These will enable three-population visualization in 1.8f.

## Parameters

In [1]:
# Model
MODEL_NAME = "Qwen3-4B-Instruct-2507"

# Input paths
REACHABILITY_PATH = f'../tensors/{MODEL_NAME}/1.8d_full_vocab_reachability.safetensors'
CLUSTER_TOKENS_PATH = f'../tensors/{MODEL_NAME}/1.4h_cluster_tokens.safetensors'

# Output
OUTPUT_PATH = f'../tensors/{MODEL_NAME}/1.8e_unreachable_tokens.safetensors'

## Imports

In [2]:
import torch
from safetensors.torch import load_file, save_file

## Load Reachability Data

In [3]:
print(f"Loading reachability data from 1.8d...\n")

reach_data = load_file(REACHABILITY_PATH)

all_unreachable = reach_data['all_unreachable']
unreachable_outside_cluster = reach_data['unreachable_outside_cluster']

print(f"✓ Loaded reachability data")
print(f"  All unreachable: {len(all_unreachable):,} tokens")
print(f"  Unreachable outside cluster: {len(unreachable_outside_cluster):,} tokens")

Loading reachability data from 1.8d...

✓ Loaded reachability data
  All unreachable: 3,357 tokens
  Unreachable outside cluster: 1,423 tokens


## Load Cluster Tokens

In [4]:
print(f"\nLoading cluster tokens from 1.4h...\n")

cluster_data = load_file(CLUSTER_TOKENS_PATH)
cluster_token_ids = cluster_data['cluster_token_ids']

print(f"✓ Loaded cluster tokens")
print(f"  Cluster size: {len(cluster_token_ids):,} tokens")


Loading cluster tokens from 1.4h...

✓ Loaded cluster tokens
  Cluster size: 2,212 tokens


## Create Vocabulary Masks

In [5]:
print(f"\nCreating boolean masks for full vocabulary...\n")

# Determine vocabulary size from max token ID
vocab_size = max(
    all_unreachable.max().item(),
    cluster_token_ids.max().item()
) + 1

print(f"Vocabulary size: {vocab_size:,} tokens\n")

# Create boolean masks
unreachable_mask = torch.zeros(vocab_size, dtype=torch.bool)
unreachable_mask[all_unreachable] = True

halo_mask = torch.zeros(vocab_size, dtype=torch.bool)
halo_mask[unreachable_outside_cluster] = True

cluster_mask = torch.zeros(vocab_size, dtype=torch.bool)
cluster_mask[cluster_token_ids] = True

# Verify mutual exclusivity (halo and cluster should be disjoint)
overlap = (halo_mask & cluster_mask).sum().item()

print(f"✓ Masks created")
print(f"  Unreachable mask: {unreachable_mask.sum().item():,} True")
print(f"  Halo mask: {halo_mask.sum().item():,} True")
print(f"  Cluster mask: {cluster_mask.sum().item():,} True")
print(f"\nValidation:")
print(f"  Halo ∩ Cluster: {overlap} tokens")
if overlap == 0:
    print(f"  ✓ Halo and cluster are disjoint (as expected)")
else:
    print(f"  ⚠️  WARNING: Halo and cluster overlap!")


Creating boolean masks for full vocabulary...

Vocabulary size: 151,936 tokens

✓ Masks created
  Unreachable mask: 3,357 True
  Halo mask: 1,423 True
  Cluster mask: 2,212 True

Validation:
  Halo ∩ Cluster: 0 tokens
  ✓ Halo and cluster are disjoint (as expected)


## Summary Statistics

In [6]:
print(f"\n{'='*70}")
print("SUMMARY: THREE POPULATIONS")
print(f"{'='*70}\n")

n_cluster = cluster_mask.sum().item()
n_halo = halo_mask.sum().item()
n_unreachable = unreachable_mask.sum().item()
n_bulk = vocab_size - n_unreachable

print(f"Population breakdown:")
print(f"  1. Cluster (tight singularity): {n_cluster:,} tokens ({100*n_cluster/vocab_size:.2f}%)")
print(f"  2. Halo (diffuse unreachables): {n_halo:,} tokens ({100*n_halo/vocab_size:.2f}%)")
print(f"  3. Bulk (healthy tokens): {n_bulk:,} tokens ({100*n_bulk/vocab_size:.2f}%)")
print(f"\nTotal unreachable: {n_unreachable:,} tokens")
print(f"  = Cluster ∩ Unreachable ({cluster_mask & unreachable_mask}.sum()) + Halo ({n_halo:,})")
print(f"  = {(cluster_mask & unreachable_mask).sum().item():,} + {n_halo:,} = {n_unreachable:,}")


SUMMARY: THREE POPULATIONS

Population breakdown:
  1. Cluster (tight singularity): 2,212 tokens (1.46%)
  2. Halo (diffuse unreachables): 1,423 tokens (0.94%)
  3. Bulk (healthy tokens): 148,579 tokens (97.79%)

Total unreachable: 3,357 tokens
  = Cluster ∩ Unreachable (tensor([False, False, False,  ..., False, False, False]).sum()) + Halo (1,423)
  = 1,934 + 1,423 = 3,357


## Save Results

In [7]:
print(f"\n{'='*70}")
print("SAVING RESULTS")
print(f"{'='*70}\n")

save_file({
    # Token IDs (sorted)
    'all_unreachable_ids': all_unreachable,
    'halo_token_ids': unreachable_outside_cluster,
    'cluster_token_ids': cluster_token_ids,
    
    # Boolean masks (full vocabulary)
    'unreachable_mask': unreachable_mask,
    'halo_mask': halo_mask,
    'cluster_mask': cluster_mask,
}, OUTPUT_PATH)

print(f"✓ Saved to {OUTPUT_PATH}")
print(f"\nSaved data:")
print(f"  Token IDs: all_unreachable_ids, halo_token_ids, cluster_token_ids")
print(f"  Masks: unreachable_mask, halo_mask, cluster_mask")


SAVING RESULTS

✓ Saved to ../tensors/Qwen3-4B-Instruct-2507/1.8e_unreachable_tokens.safetensors

Saved data:
  Token IDs: all_unreachable_ids, halo_token_ids, cluster_token_ids
  Masks: unreachable_mask, halo_mask, cluster_mask


## Summary

This notebook extracted and saved three token populations:

**1. Cluster (2,212 tokens):**
- Geometrically collapsed into tight singularity
- 87.4% unreachable, 12.6% reachable but untrained
- The "frozen core"

**2. Halo (1,423 tokens):**
- Unreachable tokens scattered in space
- NOT part of geometric cluster
- Hypothesis: escaped core via stochastic kicks but stayed in orbit

**3. Bulk (~148k tokens):**
- Normal, healthy vocabulary
- Reachable by tokenizer, trained on real data
- Escaped to live tokens region

**Next:** Visualize all three populations in 1.8f to test spatial halo hypothesis.